# FINAL PROJECT

**Introduction**

bla bla bla bla bla 

## IMPORT LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks

import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from scipy.stats.mstats import winsorize

import math
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix

from sklearn.neighbors import KNeighborsRegressor

import warnings
warnings.filterwarnings('ignore')

## IMPORT DATA

In [ ]:
def read_multiple_files(path):
    for i in path:
        if i[-4:] == 'xlsx':
            

In [19]:
data = pd.read_excel(r'Data\kickstarter_art_newest.xlsx')

## STEP 3

In [20]:
data

,backers_count,blurb,categoryId,categoryName,categorySlug,converted_pledged_amount,country,country_displayable_name,created_at,created_at_formatted,...,spotlight,staff_pick,state,state_changed_at,static_usd_rate,title,url,usd_exchange_rate,usd_pledged,usd_type
0,2,A set of unofficial merchandise consisting of ...,54,Mixed Media,art/mixed media,96,US,the United States,1635221141,"Tue, 26 Oct 2021 04:05:41 +0000",...,False,False,live,1635535014,1.000000,Genshin Impact: Idle Days (Fan Made),https://www.kickstarter.com/projects/vaileigh/...,1.000000,96.000000,international
1,2,Acrylic pins of milk cartons based on various ...,1,Art,art,38,US,the United States,1635298414,"Wed, 27 Oct 2021 01:33:34 +0000",...,False,False,live,1635523302,1.000000,Anime and Gaming Themed Milk Carton Acrylic Pins,https://www.kickstarter.com/projects/hannahrae...,1.000000,38.000000,international
2,8,"Enamel pin series inspired by frogs, rpg games...",22,Illustration,art/illustration,327,CA,Canada,1634170456,"Thu, 14 Oct 2021 00:14:16 +0000",...,False,False,live,1635523206,0.808123,Dungeons & Frogs! ✿ An Enamel Pin Adventure,https://www.kickstarter.com/projects/fredlanth...,0.809654,326.715886,international
3,10,Peanas para todos los wargames,1,Art,art,158,ES,Spain,1633948798,"Mon, 11 Oct 2021 10:39:58 +0000",...,False,False,live,1635521825,1.160569,BASE COLLECTION Vol.1,https://www.kickstarter.com/projects/rakovende...,1.167632,157.837441,international
4,11,Crappy Custom Postcard Drawings ...and poetry ...,1,Art,art,98,US,the United States,1635409716,"Thu, 28 Oct 2021 08:28:36 +0000",...,False,False,live,1635504498,1.000000,The Dumbest Thing in Your Mailbox - Part 23,https://www.kickstarter.com/projects/luckychar...,1.000000,98.000000,international
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,122,PlayStation 2 Controller & Original GameBoy So...,22,Illustration,art/illustration,2199,GB,the United Kingdom,1606424716,"Thu, 26 Nov 2020 21:05:16 +0000",...,True,False,successful,1612041864,1.368172,Retro Game Console Enamel Pins,https://www.kickstarter.com/projects/bylittlea...,1.370526,2195.916830,international
2396,18,Handmade glass mushrooms to give your plants s...,25,Sculpture,art/sculpture,762,CA,Canada,1609634750,"Sun, 03 Jan 2021 00:45:50 +0000",...,True,False,successful,1612472815,0.787146,Make 100 glass plant buddies - Handmade glass ...,https://www.kickstarter.com/projects/tykatglas...,0.781274,768.482983,international
2397,121,A Seasonal Hard Enamel Pin Project,22,Illustration,art/illustration,3188,US,the United States,1609175957,"Mon, 28 Dec 2020 17:19:17 +0000",...,True,True,successful,1612468919,1.000000,Make 100: Winter Carousel Horse,https://www.kickstarter.com/projects/lozgoddes...,1.000000,3188.000000,international
2398,107,A limited edition hard enamel pin design based...,21,Digital Art,art/digital art,2000,US,the United States,1608469256,"Sun, 20 Dec 2020 13:00:56 +0000",...,True,False,successful,1611172804,1.000000,Celestial Guardian: A Hard Enamel Pin,https://www.kickstarter.com/projects/missdoodl...,1.000000,2000.000000,international


In [21]:
data.shape

(2400, 45)